In [0]:
# generate_daily_positions.py
import random
from datetime import datetime, timedelta
import pandas as pd
import json
import pytz
import sys

# Add src to path for imports
sys.path.insert(0, '/home/user/market_risk')
from src.utils.config_loader import quick_load

# Load configuration
cfg = quick_load()
currency_pairs = cfg['currency_pairs']
base_sizes = cfg['base_sizes']
config = cfg['config']

def generate_daily_positions(date, currency_pairs, base_sizes, config):
    """
    Simulate daily positions with:
    - Random walks (positions do not change drastically day-to-day)
    - Bias by currency (EUR/USD = larger positions)
    - Occasional reversals (long → short)
    """
    
    positions = []
    
    # Get parameters from config
    long_ratio = config['position_generation']['long_ratio']
    short_ratio = config['position_generation']['short_ratio']
    flat_ratio = config['position_generation']['flat_ratio']
    daily_variation = config['position_generation']['daily_variation']
    
    for pair in currency_pairs:
        base_size = base_sizes[pair]
        
        # Random walk: daily variation from base size
        daily_change = random.uniform(-daily_variation, daily_variation)
        position_size = base_size * (1 + daily_change)
        
        # Probabilistic direction assignment
        direction = 'LONG' if random.random() < long_ratio else 'SHORT'
        
        # Chance of "zero position" (flat)
        if random.random() < flat_ratio:
            position_size = 0
            direction = 'FLAT'
        
        positions.append({
            'date': date,
            'currency_pair': pair,
            'position_size': round(position_size, 2),
            'direction': direction,
            'desk': 'FX Spot Desk',
            'generated_at': datetime.now()
        })
    
    return pd.DataFrame(positions)

positions_today = generate_daily_positions(
    datetime.now(pytz.timezone(config['timezone'])).date(),
    currency_pairs,
    base_sizes,
    config
)

# turn rows into a list[dict]
records = positions_today.to_dict(orient="records")

# serialize to JSON lines (one obj per line) or normal JSON array; pick what you want
json_lines = "\n".join(json.dumps(r, default=str) for r in records)

output_path = f"{config['storage']['volumes_base']}/{config['storage']['raw_positions']}"
file_name = f"{datetime.now().date()}_positions.json"
full_path = f"{output_path}/{file_name}"

# write directly to the Volume using dbutils.fs.put
dbutils.fs.put(full_path, json_lines, overwrite=True)